# Data Preperation for Streamlit WW2 Implementation
Our WW2 data contains attacks on individual coordinates.

In this session we want to create a nice looking web-page that let's us easily compare statistics between individual countries.

In order to do this we have to aggregate our individual coordinate-based data to per-country data.

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd

## Load Raw Data

In [ ]:
# load data on european countries
europe = pd.read_pickle("../data/europe.p").to_crs("EPSG:4326")
# contains all countries intersecting our polygon from the WW2 analysis
europe.sample(5)# not only european countries per se
# columns of interest for us: name and geometry
europe.head()


In [ ]:
# load WW2 data for target_df
# geometry contains coordinate POINTS
europe_data = pd.read_pickle("../data/gdf_europe.p").to_crs("EPSG:4326")
europe_data.head()

## Replace target coordinates by target countries polygons

In [ ]:
res_intersect_europe = gpd.overlay(europe_data, europe, how='intersection')
# Basically checks for each geometry in "europe_data" (target coordinates) 
# if intersects with geometry of "target_df" (country)
# -> Merges both dataframes and keeps country polygon instead of target coordinates
res_intersect_europe.head()

In [ ]:
# Set index to country name so we can directly insert data grouped by country name
europe.set_index("name", inplace=True, drop=False)

In [ ]:
def insert_totals(target_df, data_df):
    # get attack counts and total weight of explosives per year
    # and apply it to a given target country
    # get totals
    target_df[f"N_powerplants"] = data_df.groupby("name").size()
    target_df[f"N_green"] = data_df.loc[data_df["green"]].groupby("name").size()
    # replace NaN (no a number) values with 0
    target_df.fillna(0, inplace=True)
    n_green = data_df.groupby(["name", "green"])["name"].size().unstack()
    target_df["green_ratio"] = (n_green[True] / n_green.sum(axis=1)).fillna(0)
    return target_df

In [ ]:
europe = insert_totals(europe, res_intersect_europe)

In [ ]:
europe = europe.loc[europe["continent"] == "Europe"]

In [ ]:
def insert_fuel_types(target_df, data_df):
    for pp_type in data_df["primary_fuel"].unique():
        target_df[f"N_fuel_{pp_type}"] = data_df.loc[data_df["primary_fuel"] == pp_type].groupby("name").size()
        data_df.loc[data_df["primary_fuel"] == pp_type, f"is_{pp_type}"] = True
        data_df.loc[data_df["primary_fuel"] != pp_type, f"is_{pp_type}"] = False 
        ratio = data_df.groupby(["name", f"is_{pp_type}"]).size().unstack().fillna(0)
        target_df[f"fuel_ratio_{pp_type}"] = ratio[True] / ratio.sum(axis=1)

        production = data_df.groupby(["name", f"is_{pp_type}"])["estimated_generation_gwh_2020"].sum().unstack().fillna(0)

        target_df[f"fuel_generation_ratio_{pp_type}"] = production[True] / production.sum(axis=1)
        target_df[f"fuel_generation_total_{pp_type}"] = production[True]
    return target_df

In [ ]:
europe = insert_fuel_types(europe, res_intersect_europe).fillna(0)

# German Bundeslaender

In [ ]:
# load data containing German Bundesländer
# similarly to the europe dataset it contains polygons mapping individual Bundesländer
bl = gpd.read_file('../data/vg2500_geo84.zip').to_crs("EPSG:4326")
# rename the column that specifies the name so that is the same with the remaining data
bl.rename({"GEN": "name"}, axis=1, inplace=True)

In [ ]:
bl.head()

In [ ]:
# same as before but now we want to aggregate data for individual Bundesländer
res_intersect_bl = gpd.overlay(europe_data, bl, how='intersection')

In [ ]:
res_intersect_bl.sample(5)

In [ ]:
bl.set_index("name", inplace=True)
bl = insert_totals(bl, res_intersect_bl)
bl = insert_fuel_types(bl, res_intersect_bl)
bl

## German Landkreise
Let's go even lower than Bundesländer

In [ ]:
kreise = gpd.read_file('../data/vg2500_krs.zip').to_crs("EPSG:4326")
kreise.rename({"GEN": "name"}, axis=1, inplace=True)
kreise.sample(5)

repeat the stepts

In [ ]:
res_intersect_kreise = gpd.overlay(europe_data, kreise, how='intersection')

In [ ]:
kreise.set_index("name", inplace=True)
kreise = insert_totals(kreise, res_intersect_kreise)
kreise = insert_fuel_types(kreise, res_intersect_kreise)
kreise

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world_data = pd.read_pickle("../data/gdf_world.p").to_crs("EPSG:4326")
res_intersect_world = gpd.overlay(world_data, world, how='intersection')

In [ ]:
world.set_index("name", inplace=True)
world = insert_totals(world, res_intersect_world)
world = insert_fuel_types(world, res_intersect_world)
world

In [ ]:
world["name"] = world.index
bl["name"] = bl.index
kreise["name"] = kreise.index

In [ ]:
europe.fillna(0).to_pickle("../data/europe_pp.p")
bl.fillna(0).to_pickle("../data/bl_pp.p")
kreise.fillna(0).to_pickle("../data/kreise_pp.p")
world.fillna(0).to_pickle("../data/world_pp.p")
res_intersect_kreise.to_pickle("../data/kreise_full.p")
res_intersect_world.to_pickle("../data/world_full.p")

In [ ]:
res_intersect_kreise